In [1]:
import pandas as pd

glucose = pd.read_csv("data/GLUCOSE_training_data_final.csv")

In [2]:
glucose.head()

,experiment_id,story_id,worker_id,submission_time_normalized,worker_quality_rating,selected_sentence_index,story,selected_sentence,1_specificNL,1_specificStructured,...,8_generalStructured,9_specificNL,9_specificStructured,9_generalNL,9_generalStructured,10_specificNL,10_specificStructured,10_generalNL,10_generalStructured,number_filled_in
0,e56c7c3e-4660-40fb-80d0-052d566d676a__4,e56c7c3e-4660-40fb-80d0-052d566d676a,19,20190930,3,4,It was bedtime at our house. Two of the three ...,Finally he becomes tired and falls asleep.,The third kid continues to get out of bed and...,{The third kid}_[subject] {continues}_[verb] {...,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,2
1,b03eba12-db74-4fce-a3cc-c743948ef8c2__4,b03eba12-db74-4fce-a3cc-c743948ef8c2,2,20200115,1,4,Sara was invited to the football game. She sho...,"At the end of the game, her team won.",Sara goes to a football game >Causes/Enabl...,{Sara }_[subject] {goes }_[verb] {to }_[prepos...,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,3
2,7e7aeb03-12d8-4f02-9097-c3273a692f20__3,7e7aeb03-12d8-4f02-9097-c3273a692f20,60,20191230,1,3,Kyle was leaving to pick up his sister from wo...,He decided to check his dirty clothes basket.,Kyle realized his keys were missing >Causes/En...,{Kyle}_[subject] {realized}_[verb] {his keys}_...,...,escaped,escaped,escaped,escaped,escaped,He decided to check his dirty clothes basket f...,{He}_[subject] {decided to check}_[verb] {his ...,Someone_A decided to check Something_A (that h...,{Someone_A}_[subject] {decided to check}_[verb...,5
3,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5__4,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5,142,20191203,2,4,MY DAUGHTER HAD TO DO A SCIENCE PROJECT. SO SH...,SO CHEWING GUM CAN HELP WITH YOUR CONCENTRATIO...,My daughter has a science project >Causes/En...,{My daughter}_[subject] {has }_[verb] {a }_[pr...,...,escaped,Chewing gum can help with concentration >Resul...,{Chewing gum}_[subject] {can help}_[verb] {wit...,Chewing gum can help with concentration >Resul...,{Chewing gum}_[subject] {can help}_[verb] {wit...,escaped,escaped,escaped,escaped,7
4,b8c68631-fa42-40a0-93c7-6739402e2038__4,b8c68631-fa42-40a0-93c7-6739402e2038,19,20191213,3,4,Vivian couldn't stop thinking about that man s...,Vivian spent the rest of her life chasing that...,escaped,escaped,...,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,escaped,2


In [54]:
glucose_with_overlap = pd.read_csv("GLUCOSE_training_data_final_with_overlap_0.0.csv", index_col=0)

In [47]:
glucose_with_overlap.head()

,experiment_id,story_id,worker_id,submission_time_normalized,worker_quality_rating,selected_sentence_index,story,selected_sentence,1_specificNL,1_specificStructured,...,6_has_overlap_0.5,6_has_story_overlap_0.5,7_has_overlap_0.5,7_has_story_overlap_0.5,8_has_overlap_0.5,8_has_story_overlap_0.5,9_has_overlap_0.5,9_has_story_overlap_0.5,10_has_overlap_0.5,10_has_story_overlap_0.5
0,e56c7c3e-4660-40fb-80d0-052d566d676a__4,e56c7c3e-4660-40fb-80d0-052d566d676a,19,20190930,3,4,It was bedtime at our house. Two of the three ...,Finally he becomes tired and falls asleep.,The third kid continues to get out of bed and...,{The third kid}_[subject] {continues}_[verb] {...,...,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN
1,b03eba12-db74-4fce-a3cc-c743948ef8c2__4,b03eba12-db74-4fce-a3cc-c743948ef8c2,2,20200115,1,4,Sara was invited to the football game. She sho...,"At the end of the game, her team won.",Sara goes to a football game >Causes/Enabl...,{Sara }_[subject] {goes }_[verb] {to }_[prepos...,...,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN
2,7e7aeb03-12d8-4f02-9097-c3273a692f20__3,7e7aeb03-12d8-4f02-9097-c3273a692f20,60,20191230,1,3,Kyle was leaving to pick up his sister from wo...,He decided to check his dirty clothes basket.,Kyle realized his keys were missing >Causes/En...,{Kyle}_[subject] {realized}_[verb] {his keys}_...,...,"[True, False]",True,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN,"[True, True]",True
3,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5__4,a2bb14bd-9094-4dcc-93a5-bb93b1c308a5,142,20191203,2,4,MY DAUGHTER HAD TO DO A SCIENCE PROJECT. SO SH...,SO CHEWING GUM CAN HELP WITH YOUR CONCENTRATIO...,My daughter has a science project >Causes/En...,{My daughter}_[subject] {has }_[verb] {a }_[pr...,...,"[True, False]",False,"[True, False]",False,"[None, None]",NaN,"[True, True]",True,"[None, None]",NaN
4,b8c68631-fa42-40a0-93c7-6739402e2038__4,b8c68631-fa42-40a0-93c7-6739402e2038,19,20191213,3,4,Vivian couldn't stop thinking about that man s...,Vivian spent the rest of her life chasing that...,escaped,escaped,...,"[True, True]",False,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN,"[None, None]",NaN


In [8]:
glucose_with_overlap.columns

Index(['experiment_id', 'story_id', 'worker_id', 'submission_time_normalized',
       'worker_quality_rating', 'selected_sentence_index', 'story',
       'selected_sentence', '1_specificNL', '1_specificStructured',
       '1_generalNL', '1_generalStructured', '2_specificNL',
       '2_specificStructured', '2_generalNL', '2_generalStructured',
       '3_specificNL', '3_specificStructured', '3_generalNL',
       '3_generalStructured', '4_specificNL', '4_specificStructured',
       '4_generalNL', '4_generalStructured', '5_specificNL',
       '5_specificStructured', '5_generalNL', '5_generalStructured',
       '6_specificNL', '6_specificStructured', '6_generalNL',
       '6_generalStructured', '7_specificNL', '7_specificStructured',
       '7_generalNL', '7_generalStructured', '8_specificNL',
       '8_specificStructured', '8_generalNL', '8_generalStructured',
       '9_specificNL', '9_specificStructured', '9_generalNL',
       '9_generalStructured', '10_specificNL', '10_specificStructured

### Total number of samples

In [9]:
len(glucose_with_overlap)

65522

### Number of samples by quality rating

In [10]:
glucose_with_overlap.groupby("worker_quality_rating").experiment_id.count()

worker_quality_rating
1    18193
2    29465
3    17864
Name: experiment_id, dtype: int64

In [11]:
glucose_with_overlap.groupby("worker_quality_rating").experiment_id.count() / len(glucose_with_overlap)

worker_quality_rating
1    0.277662
2    0.449696
3    0.272641
Name: experiment_id, dtype: float64

### Number of unique stories

In [12]:
glucose_with_overlap.story.nunique()

4584

## Statistics by dimension

In [52]:
def print_statistics_by_dim(data, dim=1, threshold=0.1):
    non_escaped = data[data[f"{dim}_specificNL"] != "escaped"]
    print(f"Available samples: {len(non_escaped)}, {(len(non_escaped) * 100 / len(glucose_with_overlap)):.2f} %", end="\n\n")
    print("Available samples by worker quality rating:")
    samples_by_wqr = non_escaped.groupby("worker_quality_rating").experiment_id.count()
    print(samples_by_wqr, end="\n\n")
    print(samples_by_wqr / len(non_escaped), end="\n\n")
    print("Available samples by overlap:")
    samples_by_overlap = non_escaped.groupby([f"{dim}_has_overlap_{threshold}", f"{dim}_has_story_overlap_{threshold}"]).experiment_id.count()
    print(samples_by_overlap, end="\n\n")
    print(samples_by_overlap / len(non_escaped), end="\n\n")

In [33]:
def group_overlap_by_dim(data, dim=1, threshold=0.0):
    non_escaped = data[data[f"{dim}_specificNL"] != "escaped"]
    spec_gen_col = f"{dim}_has_overlap_{threshold}"
    spec_story_col = f"{dim}_has_story_overlap_{threshold}"
    samples_by_overlap = non_escaped.groupby([spec_gen_col, spec_story_col]).experiment_id.count()
    return samples_by_overlap.rename_axis(index={spec_gen_col: "spec_gen_overlap", spec_story_col: "spec_story_overlap"}).rename(f"dim{dim}")

In [63]:
import pathlib
import pandas as pd

from overlap import group_overlap_by_dim

threshold = 0.4
dataset = f"data/GLUCOSE_training_data_final.csv"
with_overlap_file = f"{pathlib.Path(dataset).stem}_with_overlap_{threshold}.csv"

data_with_overlap = pd.read_csv(with_overlap_file)
dim_overlap_stats_df = group_overlap_by_dim(data_with_overlap, dim=1, threshold=threshold)

for dim in range(2, 11):
    dim_overlap_stats_df = pd.merge(dim_overlap_stats_df, group_overlap_by_dim(data_with_overlap, dim=dim, threshold=threshold), left_index=True, right_index=True, how="outer")

dim_overlap_stats_df = dim_overlap_stats_df.reset_index().fillna(0)
dim_overlap_stats_df.to_csv(f"{pathlib.Path(dataset).stem}_overlap_{threshold}_stats.csv")

In [45]:
glucose_with_overlap[(glucose_with_overlap["6_specificNL"] != "escaped") & (glucose_with_overlap["6_has_overlap_0.0"] == "[False, False]") & (glucose_with_overlap["6_has_story_overlap_0.0"] == False)][["story", "6_specificNL", "6_generalNL"]].iloc[0].values

array(['Lacy and Karen got in a fight. They did not talk to each other for two days. They both got very lonely. One day, Lacy apologized. Karen apologized too so they could be friends again.',
       'Karen apologizes >Causes/Enables> Lucy accepts the apology',
       'Someone_A apoligizes to  Someone_B >Causes/Enables> Someone_B forgives Someone_A'],
      dtype=object)

In [6]:
from utils import placeholders
from collections import defaultdict
from tqdm import tqdm

placeholder_stats = {dim: defaultdict(int) for dim in range(1, 11)}

for i, row in tqdm(glucose.iterrows(), total=len(glucose)):
    for ph in placeholders:
        for dim in range(1, 11):
            words = row[f"{dim}_generalNL"].split()
            if ph in words:
                placeholder_stats[dim][ph] += 1


100%|██████████| 65522/65522 [1:03:59<00:00, 17.07it/s]


In [8]:
import json
with open("placeholder_stats.json", "w") as f:
    json.dump(placeholder_stats, f, indent=4)